In [1]:
import pandas as pd
import spacy
from collections import Counter
import re 

Einlesen von der CSV und aufspalten auf alle 4 Kategorien

In [10]:
data = pd.read_csv("20 newsgroups/20newsgroups.csv", on_bad_lines='skip', sep=';')
data.dropna()
data = data.iloc[:,1:3]

dataSplit = data.drop(data[data["group"] == 1 ].index)
dataSplit = dataSplit.drop(data[data["group"] == 2 ].index)
dataAth = data.drop(data[data["group"] != 0 ].index)
dataReli = data.drop(data[data["group"] != 3 ].index)

dataSpace = data.drop(data[data["group"] != 2 ].index)
dataGraphics = data.drop(data[data["group"] != 1 ].index)
nlp = spacy.load("en_core_web_sm")

Aufskallieren der Datensätze

In [ ]:
lenAth = len(dataAth)
lenReli = len(dataReli)
lenSpace = len(dataSpace)
lenGraphics = len(dataGraphics)

In [11]:
max_length = max(len(dataAth), len(dataReli), len(dataSpace), len(dataGraphics))
def expand(data, length):
    repeats = length // len(data)  # Anzahl der kompletten Wiederholungen
    remainder = length % len(data) # Anzahl der zusätzlichen Zeilen
    # Erstelle den erweiterten Datensatz
    expanded_data = pd.concat([data] * repeats + [data.iloc[:remainder]], ignore_index=True)
    return expanded_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Skaliere die Datensätze auf die maximale Länge
dataAth = expand(dataAth, max_length)
dataReli = expand(dataReli, max_length)
dataSpace = expand(dataSpace, max_length)
dataGraphics = expand(dataGraphics, max_length)

# Nun haben alle Datensätze die gleiche Anzahl an Zeilen (max_length)
print(len(dataAth), len(dataReli), len(dataSpace), len(dataGraphics))

983 983 983 983


                        Freq_words Generierung                                                                        
Änderung zu vorherigem Code: 
- toke.text.strip entfernt leere oder Whitespace-only Einträge
- regex entfernt alle nicht Zahlen oder Buchstaben

In [12]:
word_freqAth = Counter()
for text in dataAth["text"]:
    text = str(text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct and token.text.strip())]
    word_freqAth.update(filtered_words)

In [13]:
word_freqReli = Counter()
for text in dataReli["text"]:
    text = str(text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct and token.text.strip())]
    word_freqReli.update(filtered_words)

In [14]:
word_freqSpace = Counter()
for text in dataSpace["text"]:
    text = str(text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct and token.text.strip())]
    word_freqSpace.update(filtered_words)

In [15]:
word_freqGraphics = Counter()
for text in dataGraphics["text"]:
    text = str(text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct and token.text.strip())]
    word_freqGraphics.update(filtered_words)

Tests:

In [16]:
print(word_freqReli.most_common(2))
print(word_freqReli.get("nt"))

[('nt', 1095), ('god', 864)]
1095


Generierung der neuen most_common_words:
Notwendig sind für 500 Worte:
- Reli: 
- Ath: 
- Graphics:
- Space:

In [17]:
faktor = 1
faktorRA = 3
words = 700

freqList_Reli = []
for word, freq in word_freqReli.most_common(10000):
    freqAth = word_freqAth.get(word)
    freqSpace = word_freqSpace.get(word)
    freqGraphics = word_freqGraphics.get(word)
    if len(freqList_Reli) >= words:
        break
    if ((freqAth is None or freq >= faktorRA * freqAth) and 
        (freqSpace is None or freq >= faktorRA * freqSpace) and 
        (freqGraphics is None or freq >= faktorRA * freqGraphics)):
        freqList_Reli.append(word)


freqList_Ath = []
for word, freq in word_freqAth.most_common(10000):
    freqReli = word_freqReli.get(word)
    freqSpace = word_freqSpace.get(word)
    freqGraphics = word_freqGraphics.get(word)
    if len(freqList_Ath) >= words:
        break
    if ((freqReli is None or freq >=faktorRA * freqReli) and 
        (freqSpace is None or freq >= faktorRA * freqSpace) and 
        (freqGraphics is None or freq >= faktorRA * freqGraphics)):
        freqList_Ath.append(word)

freqList_Space = []
for word, freq in word_freqSpace.most_common(10000):
    freqAth = word_freqAth.get(word)
    freqReli = word_freqReli.get(word)
    freqGraphics = word_freqGraphics.get(word)
    if len(freqList_Space) >= words:
        break
    if ((freqAth is None or freq >= faktor * freqAth) and 
        (freqGraphics is None or freq >= faktor * freqGraphics) and 
        (freqReli is None or freq >= faktor * freqReli)):
        freqList_Space.append(word)

freqList_Graphics = []
for word, freq in word_freqGraphics.most_common(10000):
    freqAth = word_freqAth.get(word)
    freqReli = word_freqReli.get(word)
    freqSpace = word_freqSpace.get(word)
    if len(freqList_Graphics) >= words:
        break
    if ((freqAth is None or freq >= faktor * freqAth) and 
        (freqSpace is None or freq >= faktor * freqSpace) and 
        (freqReli is None or freq >= faktor * freqReli)):
        freqList_Graphics.append(word)

Datei anlegen und befüllen:

In [18]:
freqList = freqList_Ath+freqList_Reli+freqList_Graphics+freqList_Space+["groupID"]
freqListData = pd.DataFrame(columns=freqList)
freqListData.to_csv("20newsgroups_freqList.csv", index=False)
freqListData.head()

,argument,atheists,atheism,atheist,islam,fallacy,quran,theism,islamic,war,...,axis,electronics,russia,meters,projected,archivename,telescopes,meteor,contest,groupID


In [20]:
freqListData_list = []

for index, row in data.iterrows():
    text = str(row['text'])
    group = str(row['group'])
    doc = nlp(text)
    count_in_freqList = Counter(token.text.lower() for token in doc if token.is_alpha)

    haeufigkeiten = {word: count_in_freqList.get(word, 0) for word in freqList}
    haeufigkeiten['groupID'] = group
    
    # Füge die Häufigkeiten als Liste der Ergebnisse hinzu
    freqListData_list.append(haeufigkeiten)

# Übertrage die verschachtelte Liste nach dem Durchlauf in ein DataFrame
freqListData = pd.DataFrame(freqListData_list)

# Speichere das DataFrame in einer CSV-Datei
freqListData.to_csv("20newsgroups_freqList.csv", index=False)

# Zeige die ersten paar Zeilen des DataFrames
freqListData.head()


,argument,atheists,atheism,atheist,islam,fallacy,quran,theism,islamic,war,...,axis,electronics,russia,meters,projected,archivename,telescopes,meteor,contest,groupID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
